In [1]:
from openai import OpenAI
import json

client = OpenAI()

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

ChatCompletion(id='chatcmpl-92FUsMOxsVsAmCq5skbURdryRXCuv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Currently, the weather in San Francisco is sunny with a temperature of 72°F, in Tokyo it is chilly with a temperature of 10°C, and in Paris it is pleasant with a temperature of 22°C.', role='assistant', function_call=None, tool_calls=None))], created=1710323254, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f0b692a78', usage=CompletionUsage(completion_tokens=44, prompt_tokens=147, total_tokens=191))


In [27]:
from openai import OpenAI
import json

client = OpenAI()

def get_product_sales(product_name, start_date=None, end_date=None):
    """
    Get the sales for a given product within a specific date range.
    In a real scenario, fetch data from your database or sales system.
    """
    
    # Simulating data for demonstration
    return json.dumps({
        "product_name": "rephrased phrase"
    })

def run_conversation():
    messages = [{"role": "user", "content": "What's the sales for product X? given the shopify.order table"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_product_sales",
                "description": "Evaluate Ambiguous Query and provide rephrase version of the query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "product_name": {"type": "string"},
                        "start_date": {"type": "string", "format": "date", "description": "Start date in YYYY-MM-DD format"},
                        "end_date": {"type": "string", "format": "date", "description": "End date in YYYY-MM-DD format"},
                    },
                    "required": ["product_name"],
                },
            },
        }
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
    # Assuming the completion might instruct the user to provide a date range
    # due to the pre-programmed response of needing more info for an ambiguous query
    
    # For simplicity, let's assume the user then specifies the dates in a follow-up,
    # which would be captured and processed in a real scenario with additional code.
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        available_functions = {"get_product_sales": get_product_sales}
        messages.append(response_message)  # Extend conversation with assistant's reply
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                product_name=function_args.get("product_name"),
                start_date=function_args.get("start_date"),
                end_date=function_args.get("end_date"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
            
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )
        
        return second_response

response = run_conversation()

In [28]:
print(response)

ChatCompletion(id='chatcmpl-92FwQBFaXHbmf7BYcxtZSHuGXii1S', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry, I am unable to provide the sales information for product X from the Shopify order table.", role='assistant', function_call=None, tool_calls=None))], created=1710324962, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f0b692a78', usage=CompletionUsage(completion_tokens=21, prompt_tokens=57, total_tokens=78))


In [20]:
from openai import OpenAI
import json

client = OpenAI()

# Dummy schema information for a sales data table
sales_data_schema = {"required": ["Product_Name", "Date"], "optional": ["Region"]}

def evaluate_and_rephrase_query(user_query):
    """
    Evaluates the user query against the table schema to identify ambiguities.
    If ambiguities are detected, generate rephrased queries that aim to clarify them.
    """
    # Basic evaluation against the schema to identify missing required fields
    missing_required_fields = [field for field in sales_data_schema["required"] if field.lower() not in user_query.lower()]
    
    if missing_required_fields:
        # If there are missing fields, construct rephrased queries to include them
        # For brevity, we'll simulate the generation of rephrased queries
        # In reality, this would involve a call to OpenAI's API or another method of generating text
        
        rephrased_queries = []
        for field in missing_required_fields:
            prompt = f"Please clarify {field}. For example: {user_query} in June 2023."
            rephrased_queries.append(prompt)
        
        # Optionally, you could use OpenAI to refine or generate these prompts
        # Example (you should replace with an actual API call):
        # response = client.Completions.create(
        #     engine="text-davinci-003",
        #     prompt="Rephrase the following user query to include missing details: '{}'.".format(user_query),
        #     max_tokens=60,
        #     n=3  # Generate 3 rephrasings
        # )
        # rephrased_queries = [choice['text'] for choice in response.choices]
        
        return json.dumps({"ambiguous": True, "rephrased_queries": rephrased_queries})
    
    else:
        # If no required fields are missing, query is not considered ambiguous
        return json.dumps({"ambiguous": False, "message": "Query is not ambiguous."})

user_query = "What's the YoY sales?"
rephrase_response = evaluate_and_rephrase_query(user_query)
print(rephrase_response)

{"ambiguous": true, "rephrased_queries": ["Please clarify Product_Name. For example: What's the YoY sales? in June 2023.", "Please clarify Date. For example: What's the YoY sales? in June 2023."]}
